In [1]:
import requests
import io

from datetime import date, timedelta

url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-16,France,110413.0,0.0,18526.0,0.0,0.0,0.0
0,2020-04-16,China,83550.0,0.0,3345.0,0.0,0.0,0.0
0,2020-04-16,Italy,167785.0,0.0,22236.0,0.0,0.0,0.0
0,2020-04-16,Spain,180260.0,0.0,19062.0,0.0,0.0,0.0
0,2020-04-16,United States,664834.0,0.0,37060.0,0.0,0.0,0.0
0,2020-04-16,World,2106928.0,0.0,145567.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")

In [6]:
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-17,France,110413.0,108847.0,18526.0,17920.0,1566.0,606.0
0,2020-04-17,China,83550.0,83754.0,3345.0,4636.0,-204.0,-1291.0
0,2020-04-17,Italy,167785.0,168941.0,22236.0,22172.0,-1156.0,64.0
0,2020-04-17,Spain,180260.0,182816.0,19062.0,19130.0,-2556.0,-68.0
0,2020-04-17,United States,664834.0,671331.0,37060.0,33284.0,-6497.0,3776.0
0,2020-04-17,World,2106928.0,2114269.0,145567.0,145144.0,-7341.0,423.0


In [7]:
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")